In [88]:
using CSV
using JLD2
using DataFrames
using Distances
using MultivariateStats
using PlotlyJS
using UMAP
using TSne
using Statistics
using StatsBase


In [16]:
function populatePHdata(imagepath, size, att)
    df = DataFrame(ids=Int64[],times=Int64[],labels=[])
    for i in 1:size
        colname = string("feature_",att,"_",i)
        df[!,colname] = Float64[]
    end
    for i in 1:972
        id = simulation_df[i,"id"]
        time = simulation_df[i,"time"]
        label = simulation_df[i,"label"]
        imagename = string(id)*"_"*string(time)*"_img.jld2"
        V = load_object(imagepath*imagename)
        row = Any[]
        append!(row,id)
        append!(row,time)
        append!(row,label)
        for x in V
            append!(row,x)
        end
        push!(df,row)
    end
    return df

end

populatePHdata (generic function with 2 methods)

In [8]:
simulation_df = CSV.read("labels.csv", DataFrame);

In [30]:
df_vr_tumour_h0 = populatePHdata("persistence_images\\vr\\tumour_h0\\",20,"T0")
df_vr_tumour_h1 = populatePHdata("persistence_images\\vr\\tumour_h1\\",400,"T1")
df_vr_macrophage_h0 = populatePHdata("persistence_images\\vr\\macrophages_h0\\",20,"M0")
df_vr_macrophage_h1 = populatePHdata("persistence_images\\vr\\macrophages_h1\\",400,"M1")
df_dowker_mt_h0 = populatePHdata("persistence_images\\dowker\\macrophage_tumour_h0\\",400,"MT0")
df_dowker_mt_h1 = populatePHdata("persistence_images\\dowker\\macrophage_tumour_h1\\",400,"MT1")
df_dowker_mv_h0 = populatePHdata("persistence_images\\dowker\\macrophage_vessel_h0\\",400,"MV0")
df_dowker_mv_h1 = populatePHdata("persistence_images\\dowker\\macrophage_vessel_h1\\",400,"MV1")
df_dowker_tv_h0 = populatePHdata("persistence_images\\dowker\\tumour_vessel_h0\\",400,"TV0")
df_dowker_tv_h1 = populatePHdata("persistence_images\\dowker\\tumour_vessel_h1\\",400,"TV1")


,ids,times,labels,feature_TV1_1,feature_TV1_2,feature_TV1_3,feature_TV1_4
,Int64,Int64,Any,Float64,Float64,Float64,Float64
1,1,250,0,3.60749e-59,8.29969e-59,1.71574e-58,3.1873e-58
2,1,300,0,1.58676e-55,4.60146e-55,1.19869e-54,2.80541e-54
3,1,350,0,4.74845e-52,4.59123e-52,4.28239e-52,4.30385e-52
4,1,400,0,4.58209e-54,1.37013e-53,3.6802e-53,8.88064e-53
5,1,450,0,3.31305e-56,8.88734e-56,2.14181e-55,4.63771e-55
6,1,500,0,4.6854e-58,1.10876e-57,2.35749e-57,4.50434e-57
7,11,250,0,8.51172e-61,1.88014e-60,3.73179e-60,6.65637e-60
8,11,300,0,4.90397e-52,4.7434e-52,4.12462e-52,3.22456e-52
9,11,350,0,2.03058e-54,5.85924e-54,1.51877e-53,3.53692e-53


In [32]:
VR_df = hcat(df_vr_tumour_h0,
             df_vr_tumour_h1[!,4:end],
             df_vr_macrophage_h0[!,4:end],
             df_vr_macrophage_h1[!,4:end]);
Dowker_df = hcat(df_dowker_mt_h0,
                 df_dowker_mt_h1[!,4:end],
                 df_dowker_mv_h0[!,4:end],
                 df_dowker_mv_h1[!,4:end],
                 df_dowker_tv_h0[!,4:end],
                 df_dowker_tv_h1[!,4:end]
);

In [94]:
df = Dowker_df
X=Matrix(df[:,4:end])
y=df.labels;
R = Distances.pairwise(Euclidean(), X, dims=1);
mds = classical_mds(R,2)';
#mds = umap(X',2)'
#mds=UMP'
# M1 dominant points
x0 = mds[y .==0, 1]
y0 = mds[y .==0, 2]
# M2 dominant points
x1 = mds[y .==1, 1]
y1 = mds[y .==1, 2]
# plot M1 and M2 dominant points
p=PlotlyJS.plot(
    [
        PlotlyJS.scatter(
            x=x0,
            y=y0,
            mode="markers",
            marker = attr(color="#215467",size=10,opacity=.5)
        ),
        PlotlyJS.scatter(
            x=x1,
            y=y1,
            mode="markers",
            marker = attr(color="#FB8500",size=10,opacity=.8)
        )
    ],
    PlotlyJS.Layout(
        width=600,
        height=600
    )
)
#savefig(p,"dowker.svg")

"dowker.svg"

In [95]:
df = VR_df
X=Matrix(df[:,4:end])
y=df.labels;
R = Distances.pairwise(Euclidean(), X, dims=1);
mds = classical_mds(R,2)';
#mds = umap(X',2)'

# M1 dominant points
x0 = mds[y .==0, 1]
y0 = mds[y .==0, 2]
# M2 dominant points
x1 = mds[y .==1, 1]
y1 = mds[y .==1, 2]
# plot M1 and M2 dominant points
p=PlotlyJS.plot(
    [
        PlotlyJS.scatter(
            x=x0,
            y=y0,
            mode="markers",
            marker = attr(color="#215467",size=10,opacity=.5)
        ),
        PlotlyJS.scatter(
            x=x1,
            y=y1,
            mode="markers",
            marker = attr(color="#FB8500",size=10,opacity=.8)
        )
    ],
    PlotlyJS.Layout(
        width=600,
        height=600
    )
)
#savefig(p,"rips.svg")

"rips.svg"

In [55]:
#CSV.write("dowker_combined.csv",Dowker_df)
#CSV.write("vr_combined.csv",VR_df)

"vr_combined.csv"